[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yuyan124/sd_tools/blob/main/kohya_sd_scripts.ipynb)

In [ ]:
# GPU INFO
!nvidia-smi
!nvcc -V
!free -h

In [ ]:
#@title # 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 更新python到3.10
!apt-get install python3.10
!python -m pip uninstall pips
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.10 2
!sudo apt-get install python3.10-distutils
!wget https://bootstrap.pypa.io/pip/get-pip.py
!python get-pip.py
!python --version

In [ ]:
#@title Clone sd-scripts, 安装依赖
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts 
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!pip install accelerate==0.17.1 
!pip install --upgrade -r requirements.txt 

!pip install -q --pre xformers==0.0.16
!apt-get -y install -qq aria2
!pip install -q --pre triton
!pip install --upgrade lion-pytorch locon

In [ ]:
#@title Clone 下载模型
#@markdown # 模型的路径是/content/models/model.ckpt

models = {
    "sd-1.5": "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors",
    "sd-2.1-512": "https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors",
    "sd-2.1-768": "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors",
    "Anything-v3-1": "https://huggingface.co/cag/anything-v3-1/resolve/main/anything-v3-1.safetensors",
    "Anything-v3-3": "https://huggingface.co/cag/anything-v3-3/resolve/main/anything-v3-3.safetensors",
    "Anything-v4-5": "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt",
    "Chilloutmix-Ni-pruned-fp32-fix": "https://civitai.com/api/download/models/11745"    
}

#@markdown ### **预制模型选择**
model_select = "Anything-v4-5" #@param ["sd-1.5", "sd-2.1-512", "sd-2.1-768", "Anything-v3-1", "Anything-v3-3", "Anything-v4-5", "Chilloutmix-Ni-pruned-fp32-fix"]
#@markdown ### **自定义模型**
model_custom = False #@param {type:"boolean"}
if not model_custom:
  model_url = models[model_select]
else:
  #@markdown ### **自定义模型的地址：选择自定义模型，预制模型会失效**
  model_url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.safetensors" #@param {type:"string"}

if "/blob/" in model_url:
  model_url = model_url.replace("/blob/", "/resolve/")

suffix = "ckpt" if model_url.endswith(".ckpt") else "safetensors"
model = f"model.{suffix}"

# 底模型下载
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d /content/models -o {model}

In [ ]:
#@markdown # 训练参数设置
#@markdown **底模型路径**
pretrained_model = "/content/models/model.safetensors" #@param {type:"string"}
#@markdown **训练数据集路径**
train_data_dir = "/content/drive/MyDrive/ai/" #@param {type:"string"}
#@markdown **日志路径**
logging_dir = "/content/drive/MyDrive/ai/logs" #@param {type:"string"}
#@markdown **正则化数据集, 不填则不加载**
reg_data_dir = "" #@param {type:"string"}

#@markdown ---
#@markdown ## 输出设置
#@markdown **模型保存名称**
output_name="test" #@param {type:"string"}
#@markdown **模型输出路径** 
output_dir = "/content/drive/MyDrive/ai/outputs/" #@param {type:"string"}
#@markdown **模型保存格式** 
save_model_as="safetensors" #@param ["ckpt", "pt", "safetensors"]

#@markdown ---
#@markdown ## 训练相关参数
#@markdown **图片分辨率，支持非正方形，但必须是 64 倍数**
resolution="512,512" #@param {type:"string"}
#@markdown **batch size**
batch_size=1 #@param {type:"number"}
#@markdown **最大训练 epoch**
max_train_epoches=20 #@param {type:"number"}
#@markdown **每N个 epoch 保存一次**
save_every_n_epochs=1 #@param {type:"number"}
clip_skip=2 # 一般用 2
#@markdown **仅训练 unet，开启这个会牺牲效果大幅减少显存使用。6G显存可以开启**
train_unet_only=False #@param {type:"boolean"}
#@markdown **仅训练 文本编码器**
train_text_encoder_only=False #@param {type:"boolean"}
#@markdown **seed，跑测试使用**
seed="1123" #@param {type:"string"}
#@markdown **在训练中添加噪声偏移来改良生成非常暗或者非常亮的图像, (0-1), 推荐0.1**
noise_offset="0.05" #@param {type:"string"}
#@markdown **在随机打乱 tokens 时，保留前 N 个不变**
keep_tokens=0  #@param {type:"number"}

#@markdown ---
#@markdown ## Learning rate 
#@markdown **学习率**
lr="6e-5" #@param {type:"string"}
#@markdown **unet 学习率**
unet_lr="6e-5" #@param {type:"string"}
#@markdown **text encoder 学习率**
text_encoder_lr="7e-6" #@param {type:"string"}
# 
lr_scheduler="cosine_with_restarts" #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
#@markdown **仅在 lr_scheduler 为 constant_with_warmup 时需要填写这个值**
lr_warmup_steps=0 #@param {type:"number"}
#@markdown **余弦退火重启次数,仅在 lr_scheduler 为 cosine_with_restarts 时起效。** 
lr_restart_cycles=1 #@param {type:"number"}

#@markdown ---
#@markdown ## 其他设置
#@markdown **若需要从已有的 LoRA 模型上继续训练，请填写 LoRA 模型路径。**
network_weights="" #@param {type:"string"}
#@markdown **arb 最小分辨率**
min_bucket_reso=256 #@param {type:"number"}
#@markdown **arb 最大分辨率**
max_bucket_reso=1024 #@param {type:"number"}
persistent_data_loader_workers=0 # 容易爆内存，保留加载训练集的worker，减少每个 epoch 之间的停顿

#@markdown ---
#@markdown ## 优化器设置
# AdamW8bit lr="1e-4" unet_lr="1e-4" text_encoder_lr="1e-5"
# adafactor lr="1e-6"
# 使用 8bit adam 优化器节省显存，默认启用。部分 10 系老显卡无法使用，修改为 0 禁用。
optimizer = "Lion" #@param ["Lion", "AdamW8bit", "adafactor"]

#@markdown ---
#@markdown ## network 设置
#@markdown **启用locon后，network_dim 和 network_alpha才会生效**
network_module = "networks.lora" #@param ["networks.lora", "locon.locon_kohya"]
# network_dim 和 network_alpha 应当选择较小的值，比如 2~16
#@markdown **network dim 常用 4~128，不是越大越好**
network_dim=32 #@param {type:"number"}
#@markdown **network alpha**
# 用于控制network dim的权重，通常是network_dim的一半 防止下溢。默认值为 1，使用较小的 alpha 需要提升学习率。
network_alpha=32 #@param {type:"number"}

# LyCORIS 训练设置
algo="lora"  # LyCORIS network algo | LyCORIS 网络算法 可选 lora、loha。lora即为locon
#@markdown ** 类似于 network_dim 推荐为4
conv_dim=4 #@param {type:"number"}
#@markdown ** 类似于 network_alpha，可以采用与 conv_dim 一致或者更小的值**
conv_alpha=4  #@param {type:"number"}


extra_args = ""

if reg_data_dir:
    extra_args += f"--reg_data_dir {reg_data_dir}"
if train_unet_only:
    extra_args += "--network_train_unet_only "
if train_text_encoder_only:
    extra_args += "--network_train_text_encoder_only"
if network_weights:
    extra_args += f"--network_weights {network_weights}"
    
if optimizer == "Lion":
    extra_args += "--use_lion_optimizer"
elif optimizer == "AdamW8bit":
    extra_args += "--use_8bit_adam"
elif optimizer == "adafactor":
    optimizer_args = "scale_parameter=True relative_step=True warmup_init=True weight_decay=2"
    extra_args += f"--optimizer_type={optimizer} --optimizer_args={optimizer_args}"
    
if network_module == "lycoris.kohya":
    extra_args += f"--network_args conv_dim={conv_dim} conv_alpha={conv_alpha} algo={algo}"


In [ ]:
#@title 训练
%cd /content/sd-scripts/
!accelerate launch --num_cpu_threads_per_process=8 "train_network.py" \
  --enable_bucket \
  --pretrained_model=$pretrained_model \
  --train_data_dir=$train_data_dir \
  --output_dir=$output_dir \
  --logging_dir=$logging_dir \
  --resolution=$resolution \
  --max_train_epochs=$max_train_epoches \
  --learning_rate=$lr \
  --unet_lr=$unet_lr \
  --text_encoder_lr=$text_encoder_lr \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=$lr_warmup_steps \
  --lr_scheduler_num_cycles=$lr_restart_cycles \
  --network_dim=$network_dim \
  --network_alpha=$network_alpha \
  --output_name=$output_name \
  --train_batch_size=$batch_size \
  --save_every_n_epochs=$save_every_n_epochs \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --seed=$seed \
  --cache_latents \
  --clip_skip=$clip_skip \
  --prior_loss_weight=1 \
  --max_token_length=225 \
  --caption_extension=".txt" \
  --save_model_as=$save_model_as \
  --min_bucket_reso=$min_bucket_reso \
  --max_bucket_reso=$max_bucket_reso \
  --noise_offset=$noise_offset \
  --keep_tokens=$keep_tokens \
  --network_module=$network_module \
  --xformers --shuffle_caption {extra_args}

In [ ]:
#@title tensorboard 可视化
%load_ext tensorboard
%tensorboard --logdir {logging_dir}